## Create Synthetic Source Data 

In [ ]:
%pip install dbldatagen==0.3.5

In [ ]:
import dbldatagen as dg
import json
import pprint

In [ ]:
dbutils.widgets.text("metadata_file_path", "", "01 Metadata File Path")

In [ ]:
# Open json metadata and deserialize to dict
with open(dbutils.widgets.get('metadata_file_path')) as f:
    task_list = json.load(f)

# Optionally filter for specific tasks
task_list = [t for t in task_list if t['batch'] in ('bronze_01', 'bronze_02')]

pprint.pprint(task_list)

In [ ]:
# https://databrickslabs.github.io/dbldatagen/public_docs/generating_cdc_data.html

def create_dataspec(row_count, partitions):
    partitions_requested = partitions
    data_rows = row_count
    spark.conf.set("spark.sql.shuffle.partitions", "auto")
    uniqueCustomers = row_count

    dataspec = (
        dg.DataGenerator(
            spark,
            rows=data_rows,
            partitions=partitions_requested,
            randomSeedMethod="hash_fieldname",
        )
        .withIdOutput()
        .withColumn("str1", "string", template=r"\\w")
        .withColumn("str2", "string", template=r"\\w")
        .withColumn(
            "event_date",
            "date",
            begin="2020-01-01",
            end="2022-12-31",
            interval="1 day",
            random=True,
        )
    )

    return dataspec

In [ ]:
# catalog = dbutils.widgets.get('catalog')
# schema = dbutils.widgets.get('schema')
# nbr_of_tables = int(dbutils.widgets.get('nbr_of_tables'))

# spark.sql(f'use catalog {catalog}')
# spark.sql(f'create schema if not exists {schema}')

# Set partitions to 1x or 2x number of cores
dataspec = create_dataspec(row_count=1_000_000, partitions=32)
dataspec.build().createOrReplaceTempView('src_vw')

In [ ]:
for t in task_list:
  qry = f"""create or replace table {t['src_catalog']}.{t['src_schema']}.{t['src_table']} as
  select * from src_vw
  """
  spark.sql(qry)